# All S1 cells information

In [1]:
import h5py
import json
import numpy as np
import os
import sys
from matplotlib import pyplot as plt

# rootFolder = '/home/fernando/S1_netpyne'
#------------------------------------------------------------------------------  

In [2]:
rootFolder = os.getcwd()
rootFolder = rootFolder[:-5]
rootFolder

'/home/fernando/S1_netpyne'

## Cell Number distribution

In [3]:
#------------------------------------------------------------------------------
# Cells
#------------------------------------------------------------------------------
# Load 55 Morphological Names-> L1:6 L23:10 L4:12 L5:13 L6:14
# Load 207 Morpho-electrical Names-> L1:14 L23:43 L4:46 L5:52 L6:52

with open(rootFolder + '/info/anatomy/S1-cells-distributions-Rat.txt') as metype_file:
    metype_content = metype_file.read()       

MtypeNumber = {}
MEtypeNumber = {}
MtypePop = []
MEtypePop = []
popLabel = {}
N = 0
for line in metype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()
    MEtypeNumber[metype] = int(n)
    popLabel[metype] = mtype
    MtypeNumber[mtype] = int(m)

    if mtype not in MtypePop:
        MtypePop.append(mtype)
    MEtypePop.append(metype)
    
    N = N + int(n)
    
print ('Number of cells = %d' % N)

Number of cells = 31346


In [4]:
Epops = ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
             'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
             'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']
Ipops = []
for popName in MtypePop:
    if popName not in Epops:
        Ipops.append(popName)

In [30]:
StochKvcells = []
nonStochKvcells = []

for cellName in MEtypePop:
    number = 1 # same for all metype cells
    os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
    
    foldermech = os.listdir('mechanisms/')
    if 'StochKv.mod' in foldermech:
        StochKvcells.append(cellName) 
        if 100*MEtypeNumber[cellName]/MtypeNumber[popLabel[cellName]] > 25:
            print('%s %s %.0f%s %d %d' % (cellName, '1', 100*MEtypeNumber[cellName]/MtypeNumber[popLabel[cellName]],'%', MEtypeNumber[cellName], MtypeNumber[popLabel[cellName]]))
#         else:
#             print('%s %s %.0f%s %d %d' % (cellName, '2', 100*MEtypeNumber[cellName]/MtypeNumber[popLabel[cellName]],'%', MEtypeNumber[cellName], MtypeNumber[popLabel[cellName]]))
    else:
        nonStochKvcells.append(cellName)
#         print('%s %s %.0f%s %d %d' % (cellName, '0', 100*MEtypeNumber[cellName]/MtypeNumber[popLabel[cellName]],'%', MEtypeNumber[cellName], MtypeNumber[popLabel[cellName]]))


L4_DBC_bIR215 1 32% 13 40
L4_LBC_cSTUT189 1 25% 31 122
L5_DBC_bIR215 1 35% 34 96
L6_DBC_bIR215 1 32% 10 31
L6_LBC_bIR215 1 27% 127 463
L6_LBC_bSTUT213 1 27% 124 463
L6_MC_bIR215 1 43% 145 336


In [6]:
StochKvcells

['L1_HAC_cIR216',
 'L1_NGC-DA_cSTUT189',
 'L23_BP_bIR215',
 'L23_BP_dSTUT214',
 'L23_BTC_bIR215',
 'L23_DBC_bIR215',
 'L23_LBC_cSTUT189',
 'L23_NBC_cIR216',
 'L23_NGC_cSTUT189',
 'L4_BP_bIR215',
 'L4_BP_dSTUT214',
 'L4_BTC_bIR215',
 'L4_BTC_bSTUT213',
 'L4_DBC_bIR215',
 'L4_DBC_bSTUT213',
 'L4_DBC_cIR216',
 'L4_LBC_cSTUT189',
 'L4_LBC_dSTUT214',
 'L4_NBC_cIR216',
 'L4_NGC_cSTUT189',
 'L5_BP_bIR215',
 'L5_BP_dSTUT214',
 'L5_DBC_bIR215',
 'L5_DBC_bSTUT213',
 'L5_DBC_cIR216',
 'L5_LBC_cIR216',
 'L5_LBC_cSTUT189',
 'L5_LBC_dSTUT214',
 'L5_MC_bIR215',
 'L5_MC_bSTUT213',
 'L5_MC_cSTUT189',
 'L5_NBC_bIR215',
 'L5_NBC_bSTUT213',
 'L5_NBC_cIR216',
 'L5_NBC_cSTUT189',
 'L5_NBC_dSTUT214',
 'L5_NGC_cSTUT189',
 'L6_BP_bIR215',
 'L6_BP_dSTUT214',
 'L6_DBC_bIR215',
 'L6_DBC_bSTUT213',
 'L6_DBC_cIR216',
 'L6_LBC_bIR215',
 'L6_LBC_bSTUT213',
 'L6_LBC_cSTUT189',
 'L6_MC_bIR215',
 'L6_MC_bSTUT213',
 'L6_MC_cIR216',
 'L6_NBC_bIR215',
 'L6_NBC_bSTUT213',
 'L6_NBC_cIR216',
 'L6_NBC_cSTUT189',
 'L6_NBC_dSTUT

In [5]:
## https://bbp.epfl.ch/nmc-portal/assets/documents/static/Download/hoc_combos_syn.1_0_10.allzips.tar
## extracted 1035 folders in home/fernando/S1_BBP/cell_data/  #~ not inclued in the github

copyStochKv_deterministic = False
compilemods = False

StochKvcells = []
nonStochKvcells = []

for cellName in MEtypePop:
    number = 1 # same for all metype cells
    os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
    
    foldermech = os.listdir('mechanisms/')
    if 'StochKv.mod' in foldermech:
        StochKvcells.append(cellName) 
    else:
        nonStochKvcells.append(cellName) 

## https://www.opensourcebrain.org/projects/blue-brain-project-showcase/repository/revisions/master/raw/NMC/NEURON/test/StochKv_deterministic.mod           
## saved in /home/fernando/S1_BBP/mod/  #~ not inclued in the github

os.chdir('/home/fernando/S1_BBP/mod/')   

if copyStochKv_deterministic:
    for cellName in MEtypePop[0:207]:
        if cellName in StochKvcells:
            for number in range(1,6):
                modfile = 'StochKv_deterministic.mod'
                outmodfile = '/home/fernando/S1_BBP/cell_data/' + cellname+'_'+str(number) + '/mechanisms/StochKv_deterministic.mod'
                os.popen("cp {0} {1}".format(modfile, outmodfile))
                print(outmodfile)

if compilemods:
    for cellName in MEtypePop[0:207]:
        for number in range(1,6):
            os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
            !nrnivmodl mechanisms # run the 1035 cells will take several minutes!!!


## StochKv channels

In [6]:
def loadTemplateName(cellName,number): 
    f = open('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/template.hoc', 'r')
    for line in f.readlines():
        if 'begintemplate' in line:
            templatename = str(line)     
    templatename=templatename[:-1]        
    templatename=templatename[14:]
    return templatename

In [7]:
def loadCell(cellName,cellTemplateName,number):
    
    from neuron import h
    
    os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
    h.load_file("stdrun.hoc")
    h.load_file('import3d.hoc')
    h.load_file("template.hoc")
    
    cell = getattr(h, cellTemplateName)(0)
    
    i=0
    for secs in cell.somatic:
        sec = cell.soma[i]
        listmech = list(cell.soma[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.basal:
        sec = cell.dend[i]
        listmech = list(cell.dend[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.apical:
        sec = cell.apic[i]
        listmech = list(cell.apic[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1

    i=0
    for secs in cell.axonal:
        sec = cell.axon[i]
        listmech = list(cell.axon[i](0.5))      
        for mech in listmech:
            if str(mech) == 'StochKv':
                print (sec, mech, i)
        i=i+1     
    
    print (cell)
    return cell

In [8]:
cellName = StochKvcells[0]
number = 1
cellTemplateName = loadTemplateName(cellName,number)
print(cellName,number,cellTemplateName)
os.chdir('/home/fernando/S1_BBP/cell_data/'+cellName+'_'+str(number)+'/')
loadCell(cellName,cellTemplateName,number)

L1_HAC_cIR216 1 cIR216_L1_HAC_84f0f8f321
	1 
	1 
	1 
cIR216_L1_HAC_84f0f8f321[0].soma[0] StochKv 0
cIR216_L1_HAC_84f0f8f321[0].dend[0] StochKv 0
cIR216_L1_HAC_84f0f8f321[0].dend[1] StochKv 1
cIR216_L1_HAC_84f0f8f321[0].dend[2] StochKv 2
cIR216_L1_HAC_84f0f8f321[0].dend[3] StochKv 3
cIR216_L1_HAC_84f0f8f321[0].dend[4] StochKv 4
cIR216_L1_HAC_84f0f8f321[0].dend[5] StochKv 5
cIR216_L1_HAC_84f0f8f321[0].dend[6] StochKv 6
cIR216_L1_HAC_84f0f8f321[0].dend[7] StochKv 7
cIR216_L1_HAC_84f0f8f321[0].dend[8] StochKv 8
cIR216_L1_HAC_84f0f8f321[0].dend[9] StochKv 9
cIR216_L1_HAC_84f0f8f321[0].dend[10] StochKv 10
cIR216_L1_HAC_84f0f8f321[0].dend[11] StochKv 11
cIR216_L1_HAC_84f0f8f321[0].dend[12] StochKv 12
cIR216_L1_HAC_84f0f8f321[0].dend[13] StochKv 13
cIR216_L1_HAC_84f0f8f321[0].dend[14] StochKv 14
cIR216_L1_HAC_84f0f8f321[0].dend[15] StochKv 15
cIR216_L1_HAC_84f0f8f321[0].dend[16] StochKv 16
cIR216_L1_HAC_84f0f8f321[0].dend[17] StochKv 17
cIR216_L1_HAC_84f0f8f321[0].dend[18] StochKv 18
cIR216_L1

cIR216_L1_HAC_84f0f8f321[0]

In [17]:
StochKvcellsNumber = 0
for metype in StochKvcells:    
    StochKvcellsNumber = StochKvcellsNumber + MEtypeNumber[metype]

print('cells with StochKv channel = %d (%.2f percent) ' % (StochKvcellsNumber,100.0*StochKvcellsNumber/N))     

cells with StochKv channel = 1137 (3.63 percent) 


## Cell distribution in the cylinder

In [9]:
def volume(sizey, radius):    
    sizey = 0.001 * sizey # from um to mm
    radius = 0.001 * radius # from um to mm    
    vol = np.pi * radius**2 * sizey
    return vol

def cellNumber(sizey, radius, density):
    number = volume(sizey, radius) * density
    return number

In [10]:
# RAT Cell 2015 
Layerthicknesses = {}  
Layerthicknesses['L1'] = 165
Layerthicknesses['L2'] = 149
Layerthicknesses['L3'] = 353
Layerthicknesses['L4'] = 190
Layerthicknesses['L5'] = 525
Layerthicknesses['L6'] = 700
Neurondensities = {}
Neurondensities['L1'] = 14200
Neurondensities['L2'] = 164600
Neurondensities['L3'] = 83800
Neurondensities['L4'] = 177300
Neurondensities['L5'] = 83900
Neurondensities['L6'] = 131500
neuronsperlayer = {}
neuronsperlayer['L1'] = 338
neuronsperlayer['L23'] = 7524
neuronsperlayer['L4'] = 4656
neuronsperlayer['L5'] = 6114
neuronsperlayer['L6'] = 12651

radius = 210
for layer in ['L1','L4','L5','L6']:
    print ('cell Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])))    
    print ('comparation BBPwebsite %.3f' % (cellNumber(Layerthicknesses[layer], radius, Neurondensities[layer])/neuronsperlayer[layer]))

print ('cell Number in Layer L23 =  %.0f' % (cellNumber(Layerthicknesses['L2'], radius, Neurondensities['L2']) + cellNumber(Layerthicknesses['L3'], 210, Neurondensities['L3'])))      
print ('comparation BBPwebsite %.3f' % ((cellNumber(Layerthicknesses['L2'], radius, Neurondensities['L2']) + cellNumber(Layerthicknesses['L3'], 210, Neurondensities['L3']))/neuronsperlayer['L23']))

synapticdensities = {}
synapticdensities['L1'] = 61728000.0
synapticdensities['L2'] = 101313777.77778
synapticdensities['L3'] = 101313777.77778
synapticdensities['L4'] = 351032889.0
synapticdensities['L5'] = 350586074.0
synapticdensities['L6'] = 105211259.0
synapticNumberMouse = 0
synapticNumberperLayerMouse = {}
for layer in ['L1','L2','L3','L4','L5','L6']:
    print ('synaptic Number in Layer %s =  %.0f' % (layer,cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])))       
    synapticNumberperLayerMouse[layer] = cellNumber(Layerthicknesses[layer], radius, synapticdensities[layer])
    synapticNumberMouse = synapticNumberMouse + synapticNumberperLayerMouse[layer]
    
print ('synaptic Number in a cilinder (d = %d um) =  %.0f' % (2*radius,synapticNumberMouse))   


cell Number in Layer L1 =  325
comparation BBPwebsite 0.960
cell Number in Layer L4 =  4667
comparation BBPwebsite 1.002
cell Number in Layer L5 =  6103
comparation BBPwebsite 0.998
cell Number in Layer L6 =  12753
comparation BBPwebsite 1.008
cell Number in Layer L23 =  7496
comparation BBPwebsite 0.996
synaptic Number in Layer L1 =  1411090
synaptic Number in Layer L2 =  2091430
synaptic Number in Layer L3 =  4954863
synaptic Number in Layer L4 =  9240381
synaptic Number in Layer L5 =  25500132
synaptic Number in Layer L6 =  10203489
synaptic Number in a cilinder (d = 420 um) =  53401385


In [11]:
sizeY = 0
for layer in ['L1','L2','L3','L4','L5','L6']:
    sizeY = sizeY + Layerthicknesses[layer]
sizeY

2082

In [12]:
volume(sizeY,radius)

0.28844909940053115

In [13]:
# Load 55 Morphological Names and Cell pop numbers -> L1:6 L23:10 L4:12 L5:13 L6:14
# Load 207 Morpho-electrical Names used to import the cells from 'cell_data/' -> L1:14 L23:43 L4:46 L5:52 L6:52
RatL1Number = 0
for popName in MtypePop[0:6]:
    RatL1Number = RatL1Number + MtypeNumber[popName]
RatL23Number = 0
for popName in MtypePop[6:16]:
    RatL23Number = RatL23Number + MtypeNumber[popName]
RatL4Number = 0
for popName in MtypePop[16:28]:
    RatL4Number = RatL4Number + MtypeNumber[popName]
RatL5Number = 0
for popName in MtypePop[28:41]:
    RatL5Number = RatL5Number + MtypeNumber[popName]    
RatL6Number = 0
for popName in MtypePop[41:55]:
    RatL6Number = RatL6Number + MtypeNumber[popName]  

In [14]:
cellNumberperLayerRat = {}
cellNumberperLayerRat['L1'] = RatL1Number
cellNumberperLayerRat['L23'] = RatL23Number
cellNumberperLayerRat['L4'] = RatL4Number
cellNumberperLayerRat['L5'] = RatL5Number
cellNumberperLayerRat['L6'] = RatL6Number
    
RatL1Number + RatL23Number + RatL4Number + RatL5Number + RatL6Number

31346

In [19]:
ExcNumber = 0
for mtype in Epops:    
    ExcNumber = ExcNumber + MtypeNumber[mtype]
    print ('%d %s cells' % (MtypeNumber[mtype],mtype))

print('\n Exc cells = %d (%.2f percent) ' % (ExcNumber,100.0*ExcNumber/N))     

5877 L23_PC cells
2674 L4_PC cells
406 L4_SS cells
1098 L4_SP cells
2403 L5_TTPC1 cells
2003 L5_TTPC2 cells
302 L5_STPC cells
342 L5_UTPC cells
1637 L6_TPC_L1 cells
1440 L6_TPC_L4 cells
3174 L6_BPC cells
3476 L6_IPC cells
1735 L6_UTPC cells

 Exc cells = 26567 (84.75 percent) 
